In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
import os

import pymysql
from warnings import filterwarnings

_connection = None

def get_connection(db_config):
    """
    get db connection
    :return:
    """
    global _connection
    if _connection is None:
        _connection = pymysql.connect(host=db_config['host'], user=db_config['username'],
                                      password=db_config['password'],
                                      db=db_config['db'], charset="utf8")
        filterwarnings('ignore', category=pymysql.Warning)

    return _connection


def close():
    """
    close DB connection
    :return:
    """
    global _connection
    if _connection is not None:
        _connection.close()
    _connection = None

In [3]:
db = {
    'host': '172.26.187.242',
    'username': 'malware_r',
    'password': 'GEg22v2O7jbfWhb3',
    'db': 'malware'
}

In [4]:
import time

def get_specific_data(table_suffix, sql=None):
    start_time = time.time()
    
    global _connection
    if _connection is None:
        raise Exception("please init db connect first")

    cursor = _connection.cursor()
    cursor.execute("SET NAMES utf8mb4")

    ret = []
   
    cursor.execute(sql)

    field_names = [i[0] for i in cursor.description]

    for row in cursor:
        temp = {}
        for key in range(len(row)):
            temp[field_names[key]] = row[key]
        ret.append(temp)
     
    cursor.close()
    # _connection.close()
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return ret

In [5]:
close()
res1 = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
for suffix in table_suffix:
    sql6 = """
select
  a.mw_file_hash,
  a.mw_num_engines,
  a.mw_file_prefix,
  a.mw_file_suffix,
  section_name,
  virtual_size,
  pointerto_raw_data
from mw_index_2017_%s a
  left join mw_index_2017_section_%s b on a.mw_file_hash = b.mw_file_hash
where CNT_CODE = 1 and MEM_EXECUTE = 1 and mw_num_engines <> -1 and (mw_num_engines > 6 or mw_num_engines = 0)
  and mw_file_prefix in ('201701')
    """ % (suffix, suffix)
    res1.extend(get_specific_data(suffix, sql6))
close()
print(len(res1))

--- 0.70159912109375 seconds ---
--- 0.6680092811584473 seconds ---
--- 0.6432478427886963 seconds ---
--- 0.6327581405639648 seconds ---
--- 0.6401398181915283 seconds ---
--- 0.6680278778076172 seconds ---
--- 0.675234317779541 seconds ---
--- 0.6311783790588379 seconds ---
--- 0.7162714004516602 seconds ---
--- 0.6555304527282715 seconds ---
--- 0.5963871479034424 seconds ---
--- 0.5353579521179199 seconds ---
--- 0.45333290100097656 seconds ---
--- 0.43079423904418945 seconds ---
--- 0.4037284851074219 seconds ---
--- 0.4111042022705078 seconds ---
274812


In [6]:
close()
res2 = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
for suffix in table_suffix:
    sql6 = """
select
  a.mw_file_hash,
  a.mw_num_engines,
  a.mw_file_prefix,
  a.mw_file_suffix,
  section_name,
  virtual_size,
  pointerto_raw_data
from mw_index_2017_%s a
  left join mw_index_2017_section_%s b on a.mw_file_hash = b.mw_file_hash
where CNT_CODE = 1 and MEM_EXECUTE = 1 and mw_num_engines <> -1 and (mw_num_engines > 6 or mw_num_engines = 0)
  and mw_file_prefix in ('201702')
    """ % (suffix, suffix)
    res2.extend(get_specific_data(suffix, sql6))
close()
print(len(res2))

--- 0.4007298946380615 seconds ---
--- 0.3579287528991699 seconds ---
--- 0.3581352233886719 seconds ---
--- 0.32486748695373535 seconds ---
--- 0.3375849723815918 seconds ---
--- 0.37219715118408203 seconds ---
--- 0.40309906005859375 seconds ---
--- 0.3790857791900635 seconds ---
--- 0.3896019458770752 seconds ---
--- 0.4195430278778076 seconds ---
--- 0.4056985378265381 seconds ---
--- 0.37197399139404297 seconds ---
--- 0.33071446418762207 seconds ---
--- 0.34854578971862793 seconds ---
--- 0.38054347038269043 seconds ---
--- 0.3898921012878418 seconds ---
135306


In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import pylab as pl
from sklearn.model_selection import train_test_split

max_length = 10000

train_data = pd.DataFrame(res1)
# train_data = train_data.loc[train_data.virtual_size <= max_length]
# train_data = train_data.reset_index(drop=True)
train_data.mw_num_engines[train_data.mw_num_engines == 0 ] = 0
train_data.mw_num_engines[train_data.mw_num_engines > 6 ] = 1
train_label = train_data.mw_num_engines.ravel()

test_data = pd.DataFrame(res2)
# test_data = test_data.loc[test_data.virtual_size <= max_length]
# test_data = test_data.reset_index(drop=True)
test_data.mw_num_engines[test_data.mw_num_engines == 0 ] = 0
test_data.mw_num_engines[test_data.mw_num_engines > 6 ] = 1
test_label = test_data.mw_num_engines.ravel()

x_train, x_val, y_train, y_val = train_test_split(train_data, train_label, test_size=0.1, random_state=2345)
x_test = test_data
y_test = test_label

x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ip

In [8]:
max_length = 10000

In [9]:
import keras
import numpy as np

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'

    def __init__(self, list_IDs, datasets, labels, batch_size=32, dim=8192, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.datasets = datasets
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'  # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size, self.dim), dtype=float)
        y = np.zeros(self.batch_size, dtype=float)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            base_path = "/ssd/2017/{0}/{1}{2}"
            item = self.datasets.loc[ID]
            file_path = base_path.format(item["mw_file_prefix"], item["mw_file_hash"], item["mw_file_suffix"])
            in_file = open(file_path, 'rb')
            in_file.seek(item['pointerto_raw_data'])
            if item['virtual_size'] > max_length:
                bytes_data = [int(single_byte) for single_byte in in_file.read(max_length)]
            else:
                bytes_data = [int(single_byte) for single_byte in in_file.read(item['virtual_size'])]
            X[i, 0:len(bytes_data)] = bytes_data
            y[i] = self.labels[ID]

        return X, y

Using TensorFlow backend.


In [10]:
import os
from warnings import filterwarnings

import keras
import lightgbm as lgb
import pymysql
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.layers import *
from keras.models import Model
from keras.optimizers import RMSprop
from sklearn import metrics as sm
import pandas as pd


class XX_DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'

    def __init__(self, list_IDs, datasets, batch_size=32, dim=300000, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.datasets = datasets
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, X = self.__data_generation(list_IDs_temp)

        return X, X

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'  # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size, self.dim), dtype=float)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            base_path = "/ssd/2017/{0}/{1}{2}"
            item = self.datasets.loc[ID]
            file_path = base_path.format(item["mw_file_prefix"], item["mw_file_hash"], item["mw_file_suffix"])
            in_file = open(file_path, 'rb')
            in_file.seek(item['pointerto_raw_data'])
            if item['virtual_size'] > max_length:
                bytes_data = [int(single_byte) for single_byte in in_file.read(max_length)]
            else:
                bytes_data = [int(single_byte) for single_byte in in_file.read(item['virtual_size'])]
            X[i, 0:len(bytes_data)] = bytes_data

        X = X.reshape((-1, 400, 750, 1)) / 255.0
        return X, X

class Autoencoder():
    def __init__(self, autoencoder_name, encoder_name, input_shape, conv_filter_num, conv_kernel_size, pooling_size,
                 dense_num):
        self.autoencoder_name = autoencoder_name
        self.encoder_name = encoder_name
        self.input_shape = input_shape
        self.conv_filter_num = conv_filter_num
        self.conv_kernel_size = conv_kernel_size
        self.pooling_size = pooling_size
        self.dense_num = dense_num
        self.autoencoder = None
        self.encoder = None

    def get_model(self):
        input_img = Input(self.input_shape)

        x = Conv2D(self.conv_filter_num, self.conv_kernel_size, activation='relu', padding='same')(input_img)
        x = MaxPooling2D(self.pooling_size, padding='same')(x)
        encoded = Dense(self.dense_num, activation='sigmoid')(x)

        x = Conv2D(self.conv_filter_num, self.conv_kernel_size, activation='relu', padding='same')(encoded)
        x = UpSampling2D(self.pooling_size)(x)
        decoded = Conv2D(1, self.conv_kernel_size, activation='sigmoid', padding='same')(x)

        self.autoencoder = Model(inputs=input_img, outputs=decoded)
        self.encoder = Model(inputs=input_img, outputs=encoded)

        self.autoencoder.compile(loss='mean_squared_error', optimizer=RMSprop())
        self.autoencoder.summary()

    def train(self, train_df, max_epoch, batch_size=16):
        self.get_model()
        partition_train, partition_validation = train_test_split(range(len(train_df)), test_size=0.05,
                                                                 random_state=1234)
        print('Length of the train: ', len(partition_train))
        print('Length of the validation: ', len(partition_validation))

        #         tensor_board = TensorBoard(log_dir='./logs/', batch_size=batch_size)
        file_path = "/home/zhaoqi/models/{epoch:04d}-{val_loss:.5f}.h5"
        #         early_stopping = EarlyStopping("val_loss", patience=2, verbose=0, mode='auto')
        check_point = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
        callbacks_list = [check_point]

        # Generators
        training_generator = XX_DataGenerator(partition_train, train_df, batch_size, 300000)
        validation_generator = XX_DataGenerator(partition_validation, train_df, batch_size, 300000)

        self.autoencoder.fit_generator(generator=training_generator,
                                       validation_data=validation_generator,
                                       use_multiprocessing=True,
                                       epochs=max_epoch,
                                       workers=6,
                                       callbacks=callbacks_list)
        self.autoencoder.save(self.autoencoder_name)
        self.encoder.save(self.encoder_name)

    def predict(self, test_df, batch_size=32):
        testing_generator = XX_DataGenerator(range(len(test_df)), test_df, batch_size, 300000)
        encoder_output = self.encoder.predict_generator(generator=testing_generator, max_queue_size=10, workers=6,
                                                        use_multiprocessing=True, verbose=1)
        return encoder_output


In [ ]:
autoencoder = Autoencoder('autoencoder.h5', 'encoder.h5', (400, 750, 1,), 16, 4, 25, 1)
autoencoder.train(x_train, max_epoch=1, batch_size=16)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 400, 750, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 400, 750, 16)      272       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 30, 16)        0         
_________________________________________________________________
dense_1 (Dense)              (None, 16, 30, 1)         17        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 30, 16)        272       
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 400, 750, 16)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 400, 750, 1)       257       
Total para

Process ForkPoolWorker-3:
Process ForkPoolWorker-4:
Process ForkPoolWorker-6:
Process ForkPoolWorker-1:
Process ForkPoolWorker-5:
Traceback (most recent call last):
  File "/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/keras/utils/data_utils.py", line 401, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "<ipython-input-10-74e84ab83b36>", line 40, in __getitem__
    X, X = self.__data_generation(list_IDs_temp)
  File "<ipython-input-10-74e84ab83b36>", line 60, in __data_generation
    in_file = open(file_path, 'rb')
KeyboardInterrupt
Traceback (most recent call last):